In [1]:
#spark
__author__ = 'vcoder'

import os
import sys

#change them when spark and pyspark path changes
SPARK_HOME = "/home/vcoder/EDA/spark-1.5.0"
SPARK_HOME_PYTHON = SPARK_HOME + "/python"

os.environ['SPARK_HOME'] = SPARK_HOME
sys.path.append(SPARK_HOME_PYTHON)

from pyspark import SparkContext
from pyspark import SparkConf

In [2]:
import geoip2.database
reader = geoip2.database.Reader('GeoLite2-City.mmdb')

In [3]:
import pymongo
from pymongo import MongoClient
client = MongoClient()

In [4]:
sc = SparkContext('local', 'ip2geo')
raw = sc.textFile('csv/output.csv')
ipAndGeo = raw.map(lambda x: x.split(',')).map(lambda x: ((x[0],reader.city(x[0].name)), (x[1],reader.city(x[1].name))))

In [5]:
print ipAndGeo.collect()[0:1]

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: java.net.ConnectException: Call From vcoder/127.0.0.1 to localhost:54310 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:57)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:526)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:783)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:730)
	at org.apache.hadoop.ipc.Client.call(Client.java:1351)
	at org.apache.hadoop.ipc.Client.call(Client.java:1300)
	at org.apache.hadoop.ipc.ProtobufRpcEngine$Invoker.invoke(ProtobufRpcEngine.